# Notes about this lecture
Because of the Corona virus outbreak, this lecture will not be held in the classroom but online only. Further, the lecture will only be available in this written form. In order to offer support for the students we will use the gitlab issue tracker as a question & answer forum: https://git.ee.ethz.ch/python-for-engineers/class-fs20-forum and individual videoconference sessions when needed.

## Software

### Necessary software
Please install the following tools:
* python3 (https://www.python.org/downloads/ version 3.8.2 is fine.
Python is a prerequisite for jupyter)
* jupyter-notebook (https://jupyter.org/install.html)
* **Hint for Windows and OSX**: Try to install conda or miniconda (https://docs.conda.io/en/latest/miniconda.html) first. This will install Python and jupyter-notebook automatically.

### Optional (but highly recommended) software
* git (https://git-scm.com/download/). Git is harder to install but not strictly necessary. **Hint**: On Windows Git will automatically install a Linux compatible shell which can then be found as 'Git BASH'.
* If git is not available, solutions shall be uploaded on https://polybox.ethz.ch instead and the folder shall be shared with the lecturers. 

## Support
**For any issues please use the forum** at: https://git.ee.ethz.ch/python-for-engineers/class-fs20-forum and follow the instructions therein. In case of need, we will open a room using Jitsi or BigBlueButton and share the audio, video or the screen: make sure you have a microphone and speakers functioning. 

This service is offered only **during the normal lecture hours**.

# Obtaining the material for this lecture
### If git is available on your system (preferred option)
Pull the new material from the upstream repository:

```bash
cd class-fs20
git pull upstream master
```

Then launch the jupyter-notebook and open the Lecture_XX file:

```bash
anaconda # Only on ETH computers to load the Python environment.
jupyter-notebook &
```

### If git is **not** available on your system
Download the latest material from:
https://git.ee.ethz.ch/python-for-engineers/class-fs20/-/archive/master/class-fs20-master.zip
and unpack it on your computer.

# Refreshing previous lectures

Please open the jupyter-notebook of the past lecture and read through it. This will help fixing the learned notions into the long-term memory.

**Also make sure to refresh the lecture 10 "Asyncio & Networking Part 1". It is a prerequisite for this lecture.**

### ✏️ $\mu$-exercise

After having refreshed the past lecture, switch to the Exercise notebook and complete $\mu$-exercise **0**.

# Networking with `asyncio` (part 2)

## Data serialization

When data is sent over the network or written to a file it must be transformed or *serialized* into a sequence of bytes. On the side of the receiver the bytes must be correctly interpreted or *deserialized* to reconstruct the original data. In practice it is often convenient to be able to serialize structured data types such as lists, dicts or even objects. There are a myriad of possible ways to serialize an object.

A very common way for serializing base types, lists and dicts is the JSON (JavaScript Object Notation) format. JSON encodes data as a human-readable string that resembles very strongly the Python syntax. The data types supported by JSON are limited to base types, lists and dicts. Lists and dicts can be nested. The `json` module is an integral part of Python and hence, JSON can easily be used as shown in the following code.


In [ ]:
import json

# Serialize nested data types.

# Create a dict that will be serialized.
data = {
    # This can contain base types and also lists and dicts.
    "a": 1,
    "b": 1.2,
    "c": None,
    "d": True,
    "e": [1, 2],
    "f": {1: "one"}
}


# Serialize the dict into a string.
serialized = json.dumps(data)
print("serialized =", serialized)
print("type(serialized) = ", type(serialized)) # This should be a string.

# If needed for networking the string can easily be encoded as a byte string.
serialized_bytes = serialized.encode()
assert type(serialized_bytes) == bytes

print() 

# Deserialize the string into a dict.
deserialized = json.loads(serialized)
print("deserialized =", deserialized)

print("type(deserialized) = ", type(deserialized)) # This should be a dict.

### ✏️ $\mu$-exercise
Solve $\mu$-exercise 1!

## Data serialization with `pickle`

JSON is limited to base types, lists and dicts. Sometimes it is convenient to serialize full objects. For that Python ships packages such as `pickle` or `marshal`. Both provide easy ways to serialize full objects or even functions in the case of `marshal`. **Both modules are not fully safe to use as the next section will explain.**

The following shows how an object can be serialized and deserialized using the `pickle` module.

In [ ]:
import pickle

# Create a dummy class for illustration purpose.
class MyClass:
    
    def __init__(self, attr_a, attr_b):
        self.attr_a = attr_a
        self.attr_b = attr_b
        
    def __str__(self):
        return "A({}, {})".format(self.attr_a, self.attr_b)

# Create an object.
myObject = MyClass("hello", 42)
print('Type of myObject: ', type(myObject))

# Serialize the object into a byte string.
serialized = pickle.dumps(myObject)
print("serialized =", serialized)
print('Type of serialized: ', type(serialized))

print()

# Deserialize byte string back into an object.
deserialized = pickle.loads(serialized)
# 
print("deserialized =", deserialized)

# Check that "deserialized" is indeed an object.
print('Type of deserialized: ', type(deserialized))

### ✏️ $\mu$-exercise
Solve $\mu$-exercise 2!

## Problems with `pickle`

While being convenient, the `pickle` module has it's disadvantages as stated by the [official documentation](https://docs.python.org/3/library/pickle.html):

"**Warning**

**The pickle module is not secure.** Only unpickle data you trust.

It is possible to construct malicious pickle data which will **execute arbitrary code during unpickling**. Never unpickle data that could have come from an untrusted source, or that could have been tampered with.
"

The *exercises* will look at this issue in more detail.

Further, `pickle` and `marshal` are Python specific and the formats are not supported by other programming languages.

As a rule of thumb, `pickle` and `marshal` should not be used in productive software. Their use is justified only in cases where the data is fully trusted. For example if the data has been generated by the same program instance that also interprets it.

## Other serialization packages

There are also other ways to serialize data that are not covered in this lecture but explained or used in the exercises such as the following:

* `struct` https://docs.python.org/3/library/struct.html
* `numpy` https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.tobytes.html

## HTTP with `aiohttp`
The *Hyper Text Transfer Protocol* or *HTTP* is a protocol that builds up on TCP. HTTP is the protocol used for transfering websites from the server to the browser. HTTP*S* adds a layer of security by encryption around the unencrypted HTTP.

Instead of sending only payload data over the TCP connection, HTTP first sends a few lines of meta information called *header*. For example, the header allows the browser to tell the server which page to send. This works by writing a part of the URL into the header. Also HTTP defines multiple commands such as *GET* for getting data or *PUT* for uploading data.

HTTP in its basic form is quite easy to implement manually. Fortunately, there is the Python package `aiohttp` which simplifies writing HTTP client and server code.

The rest of the section shows by example how `aiohttp` can be used. The examples here are largely inspired by the official documentation (https://docs.aiohttp.org/en/stable/client_quickstart.html).

If the package `aiohttp` may require to be installed.

In [ ]:
# Simple HTTP client.
# This code loads a web page similar to a browser.
# Instead of generating graphical output, the source of the page is displayed.
#
# Example from https://docs.aiohttp.org/en/stable/client_quickstart.html

import aiohttp

# httpbin.org is a website that sends back information about the request.
# This includes the request headers, the arguments in the URL and the source IP address.
# The data will be formatted as JSON.
url = "https://httpbin.org/get"

# Send an HTTP GET request to load content from a web page.
async with aiohttp.ClientSession() as session:
    async with session.get(url) as resp: # Send a get request
        # Print the HTTP status. The status is part of the server response header.
        # It tells wether the GET request was successful or what kind of error has occurred.
        # The status `200` means that everything was fine.
        # The status `404` means that the page was not found. 
        # (If you like, append a few random characters to the url and observe the status code.)
        print("HTTP status:", resp.status)
        print("Response:", await resp.text())

In [ ]:
# Using GET parameters.
# Send an HTTP GET request with parameters to load content from a web page.

import aiohttp

url = "https://httpbin.org/get"

# HTTP urls can contain parameters after the '?' character.
# To load the url 'https://httpbin.org/get?parameter1=valueOfParameter1'
# the parameters can be passed to `sessions.get()` as follows.
params = {"parameter1": "valueOfParameter1"}

async with aiohttp.ClientSession() as session:
    async with session.get(url, params = params) as resp: # Send a get request
        print("HTTP status:", resp.status)
        print("Response:", await resp.text())
        
# Notice how the keys and values of `params` are encoded in the "url".

### ✏️ $\mu$-exercise
Solve $\mu$-exercise 3!

### Simple webserver with `aiohttp`
`aiohttp` als provides a simple way to create HTTP web servers. The core concept is the `web.Application` object. It makes the connection from the URL path to an `async` function that creates the page belonging to this URL.

Launch the following code and test the web server with your browser:

http://localhost:8080/


http://localhost:8080/myPath


http://localhost:8080/someOtherPath

In [ ]:
# Modified example from https://docs.aiohttp.org/en/stable/client_quickstart.html

from aiohttp import web

# Define async functions that will create the web pages.
# Later, URL paths will be connected to this functions.

async def index(request):
    text = "This is the main page!"
    return web.Response(text=text)

async def handle_myPath(request):
    text = "This is the 'myPath' page!"
    return web.Response(text=text)

async def handle_name(request):
    # Get the string after the first '/' in the URL.
    name = request.match_info.get('name', "Anonymous")
    text = "Hello, " + name
    return web.Response(text=text)
    
app = web.Application()
# Define how the different paths of the URL are handled.
# Connect URL paths to async functions.
# This example only deals with HTTP GET requests.
app.add_routes([
    web.get('/', index), # http://servername/ will be forwarded to `index()`
    web.get('/myPath', handle_myPath), # http://servername/myPath will be forwarded to `handle()`
    web.get('/{name}', handle_name), # http://servername/{name} will be forwarded to `handle_name()` where {name} is a variable.
])

await web._run_app(app) # This works in Jupyter notebook.

# Run the web app from a normal Python script:
# web.run_app(app) # This does not work in Jupyter notebook.

### ✏️ $\mu$-exercise
Solve $\mu$-exercise 4!

## Using the `Transport/Protocol` networking API for UDP (optional)
This lecture covers only the `stream` *application programming interface* (API) of `asyncio` which is a good abstraction for TCP streams. However, this is not suited for the connection-less UDP protocol. `asyncio` also exposes another API based on `Transport` and `Protocol` objects. Study the official documentation page on this API: https://docs.python.org/3/library/asyncio-protocol.html

Focus on the examples "UDP Echo Server" and "UDP Echo client":

https://docs.python.org/3/library/asyncio-protocol.html#udp-echo-server

https://docs.python.org/3/library/asyncio-protocol.html#udp-echo-client

# Exercises
Solve the exercises!